# Chatbot Finetuning
The code is referred from "Unsloth 2x faster finetuning" from the Unsloth official website

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Importing the Llama model

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Create the model with the set parameters


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=4,  # Lower rank to reduce overfitting
    lora_alpha=4,  # Lower alpha for smaller updates
    lora_dropout=0.1,  # Small dropout to prevent overfitting
    target_modules=["q_proj", "v_proj"],  # Fine-tune only critical modules
    use_rslora=True,  # Use RSLoRA for efficiency
    use_gradient_checkpointing="unsloth"
)

Prepare the data from the excel dataset according the format

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from datasets import Dataset

# Load the Excel file from Google Drive (adjust the path to match your file location)
file_path = '/content/drive/MyDrive/fine-tuning-3.xlsx'
data = pd.read_excel(file_path)

# Verify the content
data.head()

# Formatting function for Alpaca-style prompts
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(row):
    instruction = row["Instruction"]
    response = row["Output"]

    # Format the text using the Alpaca prompt template
    formatted_text = alpaca_prompt.format(instruction, response) + EOS_TOKEN

    return formatted_text

# Apply the formatting function row-wise
data["text"] = data.apply(formatting_prompts_func, axis=1)

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(data[["text"]])

# Check the first sample to verify
print(dataset[0])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat are the main causes of deafness?\n\n### Output:\nDeafness is often caused by factors such as aging, noise exposure, genetic conditions, ear infections, and certain medications.<|end_of_text|>'}


Set up the trainer ahead of model training

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/210 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Finally train the model

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 210 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 1,703,936


Step,Training Loss
1,3.022800
2,3.051200
3,3.171000
4,3.035500
5,3.137900
6,2.905800
7,2.854700
8,2.726400
9,2.668000
10,2.701500


Use the code below to test run the finetuned model

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is ASL?", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is ASL?\n\n### Output:\nASL stands for American Sign Language, which is a visual language used by the deaf and hard of hearing.<|end_of_text|>']

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Output:
Sorry, I can only answer questions about deafness, ASL, or app usage.</s> Can you explain how to do something else?</s> Sorry, I can only answer 

KeyboardInterrupt: 

Saving to my Google Drive

In [ ]:
save_directory = "/content/drive/MyDrive/lora-model"

# Save the model and tokenizer to Google Drive
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

Model and tokenizer saved to /content/drive/MyDrive/lora-model


Set it to True to test and load the LoRA adapters that was saved

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
One of the most famous and iconic tall towers in Paris is the Eiffel Tower. Standing at 324 meters (1,063 feet) tall, this wrought iron tower is a symbol of the city and a must-see attraction for tourists from all over the world.<|end_of_text|>
